# Main Notebook for RJMCMC Simulation Study (Two-Stage)
 This script orchestrates the full simulation study in a two-stage process:

 1.  **Stage 1: Sensitivity Analysis**: Runs a streamlined simulation across a
     grid of hyperparameters and delay scenarios to find the optimal settings.
     This stage **only runs the proposed RJMCMC method** to save time.
 2.  **Stage 2: Main Simulation**: Uses the optimal parameters found in Stage 1
     to run a larger-scale simulation for final performance evaluation against
     all benchmark methods.
 3.  **Stage 3: Full Analysis**: Processes the results from both stages to
     generate the sensitivity analysis figure and the final LaTeX summary table.

In [1]:
import os
import numpy as np
from tqdm import tqdm
import pickle
import itertools
from joblib import Parallel, delayed

# Import project-specific modules
from config import (
    SCENARIOS, N_REPLICATIONS, SENSITIVITY_REPLICATIONS,
    SENSITIVITY_GRID_PRIORS, DELAY_DIST_SENSITIVITY, SIGNAL_CACHE_DIR,
    MAIN_RESULTS_DIR, SENSITIVITY_RESULTS_DIR, PLOTS_DIR
)
from data_generation import generate_dataset
from methods import run_rjmcmc, run_pelt, run_binseg, run_rtacfr
from analysis import full_analysis_workflow, find_optimal_hyperparameters, load_and_process_sensitivity_results

## 1. Setup Simulation Worker

In [2]:
def simulation_worker(args):
    """A worker function to run a single simulation instance for any stage."""
    scenario_name, rep_idx, output_dir, params, run_benchmarks = args
    scenario_config = SCENARIOS[scenario_name]
    
    # Generate a unique, descriptive filename for this run's checkpoint
    filename_parts = [scenario_name.replace(' ', '_'), f"rep{rep_idx}"]
    if 'PRIOR_K_GEOMETRIC_P' in params:
        filename_parts.append(f"p{params['PRIOR_K_GEOMETRIC_P']}")
    if 'PRIOR_THETA_SIGMA' in params:
        filename_parts.append(f"s{params['PRIOR_THETA_SIGMA']}")
    if 'DELAY_DIST_NAME' in params:
        delay_short_name = params['DELAY_DIST_NAME'].split(' ')[0]
        filename_parts.append(f"delay_{delay_short_name}")
    filepath = os.path.join(output_dir, "_".join(filename_parts) + ".pkl")

    # Skip if this exact simulation has already been completed
    if os.path.exists(filepath):
        return f"Skipping existing: {filepath}"
    
    # Generate dataset, allowing for an override of the default delay distribution
    delay_dist_override = params.get('DELAY_DIST', None)
    data = generate_dataset(scenario_config, rep_idx, delay_dist_override=delay_dist_override)
    
    # Always run the proposed RJMCMC method.
    # The `run_rjmcmc` function is now memory-efficient and always returns summaries.
    rjmcmc_results = run_rjmcmc(data, 
                                p_geom=params.get('PRIOR_K_GEOMETRIC_P'), 
                                theta_sigma=params.get('PRIOR_THETA_SIGMA'))
    
    # Initialize results dictionary with the proposed method's output
    results = {
        'scenario': scenario_name,
        'rep_idx': rep_idx,
        'params': params,
        'data': data,
        'rjmcmc': rjmcmc_results
    }
    
    # Conditionally run benchmark methods if requested (for the main simulation)
    if run_benchmarks:
        rtacfr_results = run_rtacfr(data, scenario_name, rep_idx)
        pelt_results = run_pelt(data, scenario_name, rep_idx)
        binseg_results = run_binseg(data, scenario_name, rep_idx)
        results['rtacfr'] = rtacfr_results
        results['pelt'] = pelt_results
        results['binseg'] = binseg_results
    
    # Save the results to a pickle file
    with open(filepath, 'wb') as f:
        pickle.dump(results, f)
        
    return f"Completed: {filepath}"

## 2. Stage 1: Run Full Sensitivity Analysis (Proposed Method Only)

In [3]:
if __name__ == '__main__':
    os.makedirs(SENSITIVITY_RESULTS_DIR, exist_ok=True)
    
    print("--- STAGE 1: Starting Full Sensitivity Analysis ---")
    
    # Create the grid of all combinations of hyperparameters for sensitivity check
    p_geom_values = SENSITIVITY_GRID_PRIORS['PRIOR_K_GEOMETRIC_P']
    theta_sigma_values = SENSITIVITY_GRID_PRIORS['PRIOR_THETA_SIGMA']
    hyperparam_grid = list(itertools.product(p_geom_values, theta_sigma_values))
    
    tasks = []
    # Iterate over each delay distribution setting, scenario, replication, and hyperparameter combo
    for delay_name, delay_dist_obj in DELAY_DIST_SENSITIVITY.items():
        for scenario_name in SCENARIOS:
            for rep_idx in range(SENSITIVITY_REPLICATIONS):
                for p_geom, theta_sigma in hyperparam_grid:
                    params = {
                        'PRIOR_K_GEOMETRIC_P': p_geom,
                        'PRIOR_THETA_SIGMA': theta_sigma,
                        'DELAY_DIST_NAME': delay_name,
                        'DELAY_DIST': delay_dist_obj
                    }
                    # The `False` flag indicates not to run the benchmarks
                    tasks.append((scenario_name, rep_idx, SENSITIVITY_RESULTS_DIR, params, False))

    # Run all sensitivity analysis tasks in parallel using joblib
    print(f"Running {len(tasks)} sensitivity simulations...")
    Parallel(n_jobs=-1)(delayed(simulation_worker)(task) for task in tqdm(tasks, desc="Sensitivity Analysis"))
    
    print("--- STAGE 1: Sensitivity Analysis Complete ---")

--- STAGE 1: Starting Full Sensitivity Analysis ---
Running 1875 sensitivity simulations...


Sensitivity Analysis: 100%|███████████████████████████████████████████████████████| 1875/1875 [00:04<00:00, 463.44it/s]


--- STAGE 1: Sensitivity Analysis Complete ---


## 3. Stage 2: Run Main Simulation (All Methods)

In [4]:
if __name__ == '__main__':
    # First, find the optimal parameters by analyzing the results from Stage 1
    df_sens = load_and_process_sensitivity_results(SENSITIVITY_RESULTS_DIR)
    optimal_params = find_optimal_hyperparameters(df_sens)

    # Now, run the main, larger-scale simulation using only these optimal parameters
    os.makedirs(MAIN_RESULTS_DIR, exist_ok=True)
    os.makedirs(SIGNAL_CACHE_DIR, exist_ok=True)
    print("\n--- STAGE 2: Starting Main Simulation with Optimal Parameters ---")
    
    main_tasks = []
    for scenario_name in SCENARIOS:
        for rep_idx in range(N_REPLICATIONS):
            # The `True` flag indicates that benchmarks should be run
            main_tasks.append((scenario_name, rep_idx, MAIN_RESULTS_DIR, optimal_params, True))
            
    # Run all main simulation tasks in parallel using joblib
    print(f"Running {len(main_tasks)} main simulations...")
    Parallel(n_jobs=-1)(delayed(simulation_worker)(task) for task in tqdm(main_tasks, desc="Main Simulation"))
        
    print("--- STAGE 2: Main Simulation Complete ---")

Loading sensitivity results from: results/sensitivity: 100%|██████████████████████| 9115/9115 [00:11<00:00, 797.55it/s]



--- Optimal Hyperparameter Selection ---
Selected from 'Perfectly Matched Delay' scenario.
Optimal p_geom: 0.7
Optimal theta_sigma: 1.0
--------------------------------------


--- STAGE 2: Starting Main Simulation with Optimal Parameters ---
Running 50 main simulations...


Main Simulation: 100%|██████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 14462.12it/s]

--- STAGE 2: Main Simulation Complete ---


## 4. Stage 3: Run Full Analysis and Generate Final Outputs

In [5]:
if __name__ == '__main__':
    print("\n--- STAGE 3: Running Full Analysis Workflow ---")
    
    # This single function handles all analysis and generation of outputs (figures and tables)
    full_analysis_workflow()
    
    print("\n--- All simulations and analyses are complete. ---")
    pub_fig_path = os.path.join(PLOTS_DIR, "publication_figure.pdf")
    sens_fig_path = os.path.join(PLOTS_DIR, "sensitivity_analysis_heatmap_grid.pdf")
    table_path = os.path.join(PLOTS_DIR, "main_results_table.tex")
    
    print(f"\nFinal outputs have been saved in the '{PLOTS_DIR}' directory:")
    print(f"- Publication Figure: {os.path.basename(pub_fig_path)}")
    print(f"- Sensitivity Heatmap: {os.path.basename(sens_fig_path)}")
    print(f"- Main Results Table: {os.path.basename(table_path)}")


--- STAGE 3: Running Full Analysis Workflow ---


Loading sensitivity results from: results/sensitivity: 100%|█████████████████████| 9115/9115 [00:07<00:00, 1160.90it/s]


Sensitivity analysis heatmap grid saved at: plots\sensitivity_analysis_heatmap_grid.pdf

--- Optimal Hyperparameter Selection ---
Selected from 'Perfectly Matched Delay' scenario.
Optimal p_geom: 0.7
Optimal theta_sigma: 1.0
--------------------------------------



Loading main results from: results/main: 100%|██████████████████████████████████████| 100/100 [00:00<00:00, 563.95it/s]


Main results LaTeX table saved at: plots\main_results_table.tex
Publication figure saved at: plots\publication_figure.pdf

--- Analysis Complete ---

--- All simulations and analyses are complete. ---

Final outputs have been saved in the 'plots' directory:
- Publication Figure: publication_figure.pdf
- Sensitivity Heatmap: sensitivity_analysis_heatmap_grid.pdf
- Main Results Table: main_results_table.tex
